<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Scripts%20de%20detecci%C3%B3n%20de%20Video/VideoDetectionYOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Instalar las dependencias necesarias
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.8/872.8 kB 20.8 MB/s eta 0:00:00


**Para descomprimir la carpeta de los modelos**

In [3]:
# # Rutas de los archivos zip en Google Drive
# ruta_zip = '/content/drive/MyDrive/TFM/models/modelos.zip'

# # Rutas de las carpetas destino
# ruta_destino = '/content'

In [4]:
# import os
# import zipfile

# # Crear la carpeta destino si no existen
# os.makedirs(ruta_destino, exist_ok=True)

# # Descomprimir los archivos zip
# with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
#     zip_ref.extractall(ruta_destino)

# print(f'Archivos descomprimidos en {ruta_destino}')

Archivos descomprimidos en /content


**Prueba de detección de obteos en video a tiempo real**

In [ ]:
import cv2
import time
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# Definir la ruta del modelo
model_path = '/content/drive/MyDrive/TFM/models/modelos/YoloV8_n_pre.pt'

# Cargar el modelo YOLOv8 para segmentación
model = YOLO(model_path)

# Configuración
initial_frames_to_test = 10  # Número de frames iniciales para medir el tiempo de procesamiento

# Obtener el nombre del modelo sin la extensión
model_name = os.path.splitext(os.path.basename(model_path))[0]

# Definir la ruta de salida basada en el nombre del modelo
output_path = f"{model_name}.mp4"

video_path = "/content/drive/MyDrive/TFM/Cáncer Válvula Ileocecal.mp4"

# Función para medir el tiempo de procesamiento y determinar la frecuencia óptima
def determine_processing_interval(video_path):
    cap = cv2.VideoCapture(video_path)

    # Obtener la información del video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    # Medir el tiempo de procesamiento de los primeros `initial_frames_to_test` frames
    start_time = time.time()
    for _ in range(initial_frames_to_test):
        ret, frame = cap.read()
        if not ret:
            break
        # Procesar el frame con el modelo
        results = model(frame)
    processing_time = time.time() - start_time

    # Calcular el tiempo promedio por frame
    avg_processing_time_per_frame = processing_time / initial_frames_to_test

    # Calcular el intervalo de frames basado en el tiempo disponible
    max_processing_time_per_frame = 1 / fps  # Tiempo disponible por frame
    optimal_processing_interval = max(1, int(round(avg_processing_time_per_frame / max_processing_time_per_frame)))

    cap.release()

    return optimal_processing_interval

# Función para procesar video y guardar el resultado
def process_video(video_path, output_path, process_every_n_frames):
    cap = cv2.VideoCapture(video_path)

    # Obtener la información del video original
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Crear el objeto de escritura de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Procesar el frame si es necesario
        if frame_count % process_every_n_frames == 0:
            # Realizar la detección
            results = model(frame)

            # Dibujar los resultados en el frame
            annotated_frame = results[0].plot()
        else:
            # Si no procesamos este frame, usamos el frame original
            annotated_frame = frame

        # Escribir el frame con detecciones en el archivo de video
        out.write(annotated_frame)

        frame_count += 1

    cap.release()
    out.release()

# Determinar el intervalo óptimo de procesamiento
optimal_interval = determine_processing_interval(video_path)

print(f"Intervalo óptimo de procesamiento: cada {optimal_interval} frames")

# Procesar el video con el intervalo óptimo
process_video(video_path, output_path, optimal_interval)

print(f"El video procesado se guardó en: {output_path}")


Streaming output truncated to the last 5000 lines.
Speed: 1.6ms preprocess, 12.3ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 448)

0: 352x448 (no detections), 12.6ms
Speed: 1.7ms preprocess, 12.6ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 448)

0: 352x448 (no detections), 11.5ms
Speed: 1.7ms preprocess, 11.5ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 448)

0: 352x448 (no detections), 17.0ms
Speed: 1.7ms preprocess, 17.0ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 448)

0: 352x448 (no detections), 16.8ms
Speed: 1.8ms preprocess, 16.8ms inference, 2.5ms postprocess per image at shape (1, 3, 352, 448)

0: 352x448 (no detections), 10.8ms
Speed: 1.8ms preprocess, 10.8ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 448)

0: 352x448 (no detections), 13.1ms
Speed: 1.7ms preprocess, 13.1ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 448)

0: 352x448 (no detections), 17.0ms
Speed: 1.7ms prepr

In [ ]:
import shutil
import os

# Archivo de origen
src_file = output_path  # Asegúrate de que esto sea el archivo, no un directorio

# Directorio de destino
dest_dir = "/content/drive/MyDrive/TFM/Videos"

def copy_file(src, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    dest_file = os.path.join(dest_dir, os.path.basename(src))
    shutil.copy2(src, dest_file)

# Copiar el archivo de origen al directorio de destino
copy_file(src_file, dest_dir)

print(f"Archivo copiado de {src_file} a {dest_dir}")
